# Task 3: Classification of COVID19 tweets containing symptoms

In [1]:
!pip install -q transformers contractions imbalanced-learn ekphrasis

     |████████████████████████████████| 4.0 MB 28.3 MB/s 
     |████████████████████████████████| 80 kB 9.2 MB/s 
     |████████████████████████████████| 6.6 MB 59.2 MB/s 
     |████████████████████████████████| 895 kB 56.8 MB/s 
     |████████████████████████████████| 77 kB 6.7 MB/s 
     |████████████████████████████████| 596 kB 68.6 MB/s 
     |████████████████████████████████| 287 kB 42.5 MB/s 
     |████████████████████████████████| 106 kB 73.8 MB/s 
     |████████████████████████████████| 45 kB 3.2 MB/s 
     |████████████████████████████████| 53 kB 2.0 MB/s 


## 1. Import all the necessary libraries and data files

In [2]:
import numpy as np
import pandas as pd

import warnings
import torch
import torch.nn as nn
import time

from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertModel, BertTokenizerFast
from transformers import RobertaTokenizerFast, RobertaModel
from ekphrasis.classes.preprocessor import TextPreProcessor
from ekphrasis.classes.tokenizer import SocialTokenizer
from ekphrasis.dicts.emoticons import emoticons
from tqdm import tqdm
from sklearn.metrics import f1_score

warnings.filterwarnings("ignore")
pd.options.display.max_colwidth=None

In [3]:
train_filename = "train.tsv"
val_filename = "valid.tsv"

In [4]:
# Load data
train = pd.read_csv(train_filename, sep="\t")
validation = pd.read_csv(val_filename, sep="\t")

In [5]:
print(f"Shape of training data is {train.shape} and validation data is {validation.shape}")

Shape of training data is (9067, 3) and validation data is (500, 3)


In [6]:
# Train top 5 rows
train.head().style.set_caption("Task 3: Train dataset")

,tweet_id,tweet,label
0,13729,A growing number of Covid-19 patients whose symptoms were initially mild are now facing mysterious long-term neurological problems https://t.co/If2SgRduuw,Lit-News_mentions
1,12399,"Medical experts advise that symptoms of the novel coronavirus include fever, shortness of breath, and stinky smelly pits and feet 😳🤪",Lit-News_mentions
2,20056,"@drdavidsamadi Hubby/I:same symptoms n November 2019 after a weekend trip 2 Vegas where bus loads of Chinese tourists.1 day fever,3 days sore throat,several weeks of fatigue.He's healthy,I'm not: diabetes,hypertension,obese, respiratory issues @ 53. No meds/pneumonia,we believe was COVID-19",Nonpersonal_reports
3,10175,"1/x In the April 11 BC briefing Dr. Bonnie Henry had mentioned that there's now reports of neurologic complications after COVID-19 infection, even during recovery. There's now anecdotal reporting of neurological manifestations from WUHAN patients in JAMA:https://t.co/7spTyk7l2M",Lit-News_mentions
4,12179,Major study PHOSP-COVID investigates health impacts of #COVID19 on hospitalised patients including #mentalhealth & neurological problems. Find out more https://t.co/JLTrz0BA7f @OxfordHealthNHS https://t.co/jZ2kPyPqmS,Lit-News_mentions


## 2. Prepare the data - Clean & Prepare for Model

In [7]:
# Drop unwanted columns
train.drop(['tweet_id'], axis=1, inplace=True)
validation.drop(['tweet_id'], axis=1, inplace=True)

In [8]:
# Referred from: https://github.com/cbaziotis/ekphrasis

text_processor = TextPreProcessor(
    # terms that will be normalized
    normalize=['url', 'email', 'percent', 'money', 'phone', 'user',
        'time', 'url', 'date', 'number'],
    
    # terms that will be annotated
    annotate={"hashtag", "allcaps", "elongated", "repeated",
        'emphasis', 'censored'},
    fix_html=True,  # fix HTML tokens
    
    # corpus from which the word statistics are going to be used 
    # for word segmentation 
    segmenter="twitter", 
    
    # corpus from which the word statistics are going to be used 
    # for spell correction
    corrector="twitter", 
    
    unpack_hashtags=True,  # perform word segmentation on hashtags
    unpack_contractions=True,  # Unpack contractions (can't -> can not)
    spell_correct_elong=False,  # spell correction for elongated words
    
    # select a tokenizer. You can use SocialTokenizer, or pass your own
    # the tokenizer, should take as input a string and return a list of tokens
    tokenizer=SocialTokenizer(lowercase=True, emojis=False).tokenize,
    
    # list of dictionaries, for replacing tokens extracted from the text,
    # with other expressions. You can pass more than one dictionaries.
    dicts=[emoticons]
)

Word statistics files not found!
Downloading... done!
Unpacking... done!
Reading twitter - 1grams ...
generating cache file for faster loading...
reading ngrams /root/.ekphrasis/stats/twitter/counts_1grams.txt
Reading twitter - 2grams ...
generating cache file for faster loading...
reading ngrams /root/.ekphrasis/stats/twitter/counts_2grams.txt
Reading twitter - 1grams ...


In [9]:
train['clean_tweets'] = [" ".join(text_processor.pre_process_doc(tweet)) for tweet in train.tweet]
validation['clean_tweets'] = [" ".join(text_processor.pre_process_doc(tweet)) for tweet in validation.tweet]

In [10]:
# Train top 5 rows after pre-processing
train[['label', 'clean_tweets']].head()

,label,clean_tweets
0,Lit-News_mentions,a growing number of covid - <number> patients whose symptoms were initially mild are now facing mysterious long - term neurological problems <url>
1,Lit-News_mentions,"medical experts advise that symptoms of the novel coronavirus include fever , shortness of breath , and stinky smelly pits and feet 😳 🤪"
2,Nonpersonal_reports,"<user> hubby / i : same symptoms n <date> after a weekend trip <number> vegas where bus loads of chinese tourists . <number> day fever , <number> days sore throat , several weeks of fatigue . he ' s healthy , i am not : diabetes , hypertension , obese , respiratory issues @ <number> . no meds / pneumonia , we believe was <allcaps> covid </allcaps> - <number>"
3,Lit-News_mentions,"<number> /x in the <date> bc briefing dr . bonnie henry had mentioned that there ' s now reports of neurologic complications after <allcaps> covid </allcaps> - <number> infection , even during recovery . there ' s now anecdotal reporting of neurological manifestations from <allcaps> wuhan </allcaps> patients in <allcaps> jama </allcaps> : <url>"
4,Lit-News_mentions,major study <allcaps> phosp </allcaps> - <allcaps> covid </allcaps> investigates health impacts of <hashtag> covid 19 </hashtag> on hospitalised patients including <hashtag> mental health </hashtag> & neurological problems . find out more <url> <user> <url>


In [11]:
BATCH_SIZE = 32
N_EPOCHS = 5

In [12]:
# Define BERT tokenizer
tokenizer = RobertaTokenizerFast.from_pretrained("roberta-base")

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

In [13]:
# Tokenize train and validation data
train_enc_rob = tokenizer.batch_encode_plus(train.clean_tweets.to_list(), padding="longest", truncation=True, max_length=128, return_tensors="pt")
valid_enc_rob = tokenizer.batch_encode_plus(validation.clean_tweets.to_list(), padding="longest", truncation=True, max_length=128, return_tensors="pt")

In [14]:
train_enc_rob.keys()

dict_keys(['input_ids', 'attention_mask'])

In [15]:
train_enc_rob.input_ids.shape, train_enc_rob.attention_mask.shape

(torch.Size([9067, 128]), torch.Size([9067, 128]))

In [16]:
def get_dataloader_rob(encoding, target):
    data = (TensorDataset(encoding.input_ids, encoding.attention_mask, target))
    sampler = RandomSampler(data)
    dataloader = DataLoader(data, sampler=sampler, batch_size=BATCH_SIZE)
    return dataloader

In [17]:
# Transform the target variable

le = LabelEncoder()
train.label = le.fit_transform(train.label)
validation.label = le.transform(validation.label)

In [18]:
list(le.classes_)

['Lit-News_mentions', 'Nonpersonal_reports', 'Self_reports']

In [19]:
train_dataloader_rob = get_dataloader_rob(train_enc_rob, torch.tensor(train['label'].to_list()))
valid_dataloader_rob = get_dataloader_rob(valid_enc_rob, torch.tensor(validation['label'].to_list()))

In [20]:
# Sanity check that the tensors returned by the dataloader are correct
for batch in train_dataloader_rob:
    input_ids, attn_mask, target = batch
    print(input_ids.shape, attn_mask.shape, target.shape)
    break

torch.Size([32, 128]) torch.Size([32, 128]) torch.Size([32])


## 3. Model Building - Roberta

In [21]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [22]:
class ROBERTAclassifier(nn.Module):
    def __init__(self, transformer):
        super(ROBERTAclassifier, self).__init__()
        self.transformer = transformer
        self.linear_layer = nn.Linear(768, 3)
    
    def forward(self, ip_ids, attn_mask):
        op = self.transformer(input_ids=ip_ids,
                              attention_mask=attn_mask)
        return  self.linear_layer(op["pooler_output"])

In [23]:
def count_parameter(model):
    return sum(para.numel() for para in model.parameters() if para.requires_grad)

In [24]:
transformer_rob = RobertaModel.from_pretrained("roberta-base")
model_rob = ROBERTAclassifier(transformer_rob).to(device)
print(f"The model has {count_parameter(model_rob):,} trainable parameters.")

Downloading:   0%|          | 0.00/478M [00:00<?, ?B/s]

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.bias', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


The model has 124,647,939 trainable parameters.


In [25]:
# Define optimizer 
criterion_rob = torch.nn.CrossEntropyLoss()
optim_rob = torch.optim.AdamW(model_rob.parameters(), lr = 2e-5)

In [26]:
def train_model_rob(model, dataloader, clip=1.0):
    model.train()

    epoch_loss = 0
    batch_num = 0
    pred, target = [], []

    for index, batch in tqdm(enumerate(dataloader)):
        batch = tuple(row.to(device) for row in batch)
        input_ids, attn_mask, y = batch

        optim_rob.zero_grad()
        output = model(input_ids, attn_mask)
        loss = criterion_rob(output, y)
        loss.backward()

        nn.utils.clip_grad_norm_(model.parameters(), clip)
        optim_rob.step()

        epoch_loss += loss.item()
        batch_num += 1
        pred.extend(torch.argmax(output, -1).tolist())
        target.extend(y.tolist())
    
    return epoch_loss/batch_num, f1_score(target, pred, average='micro')

def evaluate_rob(model, dataloader):
    model.eval()

    epoch_loss = 0
    batch_num = 0
    pred, target = list(), list()

    for index, batch in enumerate(dataloader):
        batch = tuple(row.to(device) for row in batch)
        input_ids, attn_mask, y = batch
        
        with torch.no_grad():
            output = model(input_ids, attn_mask)
            loss = criterion_rob(output, y)
            
            epoch_loss += loss.item()
            batch_num += 1
            pred.extend(torch.argmax(output, -1).tolist())
            target.extend(y.tolist())
    
    return epoch_loss/batch_num, f1_score(target, pred, average='micro'), pred, target

In [27]:
best_valid_loss_rob = float('inf')
total_train_loss_rob, total_valid_loss_rob = list(), list()

In [28]:
for epoch in tqdm(range(N_EPOCHS)):
    train_loss, train_f1_score = train_model_rob(model_rob, train_dataloader_rob)
    total_train_loss_rob.append(train_loss)

    valid_loss, valid_f1_score, pred, target = evaluate_rob(model_rob, valid_dataloader_rob)
    total_valid_loss_rob.append(valid_loss)

    if valid_loss < best_valid_loss_rob:
        best_valid_loss_rob = valid_loss
        best_pred, best_target = pred, target
        torch.save(model_rob.state_dict(), "model_least_loss_rob.pt")
        print("\nBest Model Saved!!\n")
    
    torch.save(model_rob.state_dict(), "model_checkpoint_rob" + str(epoch) + ".pt")
    print("Checkpoint Model Saved!\n")

    print(f"Epoch: {epoch+1:02}")
    print(f"Train Total Loss: {train_loss:.3f} | Train F1 Score: {train_f1_score:.3f}")
    print(f"Valid Total Loss: {valid_loss:.3f} | Valid F1 Score: {valid_f1_score:.3f}")
    print("-"*20)

  0%|          | 0/5 [00:00<?, ?it/s]
0it [00:00, ?it/s]
1it [00:00,  1.23it/s]
2it [00:01,  1.44it/s]
3it [00:02,  1.52it/s]
4it [00:02,  1.56it/s]
5it [00:03,  1.58it/s]
6it [00:03,  1.59it/s]
7it [00:04,  1.60it/s]
8it [00:05,  1.60it/s]
9it [00:05,  1.60it/s]
10it [00:06,  1.60it/s]
11it [00:07,  1.60it/s]
12it [00:07,  1.60it/s]
13it [00:08,  1.61it/s]
14it [00:08,  1.61it/s]
15it [00:09,  1.61it/s]
16it [00:10,  1.61it/s]
17it [00:10,  1.61it/s]
18it [00:11,  1.61it/s]
19it [00:11,  1.61it/s]
20it [00:12,  1.61it/s]
21it [00:13,  1.61it/s]
22it [00:13,  1.61it/s]
23it [00:14,  1.61it/s]
24it [00:15,  1.61it/s]
25it [00:15,  1.61it/s]
26it [00:16,  1.60it/s]
27it [00:16,  1.60it/s]
28it [00:17,  1.60it/s]
29it [00:18,  1.60it/s]
30it [00:18,  1.60it/s]
31it [00:19,  1.60it/s]
32it [00:20,  1.60it/s]
33it [00:20,  1.60it/s]
34it [00:21,  1.60it/s]
35it [00:21,  1.60it/s]
36it [00:22,  1.60it/s]
37it [00:23,  1.59it/s]
38it [00:23,  1.59it/s]
39it [00:24,  1.59it/s]
40it [00:25,  1.


Best Model Saved!!



 20%|██        | 1/5 [03:16<13:04, 196.07s/it]

Checkpoint Model Saved!

Epoch: 01
Train Total Loss: 0.211 | Train F1 Score: 0.928
Valid Total Loss: 0.069 | Valid F1 Score: 0.976
--------------------



0it [00:00, ?it/s]
1it [00:00,  1.35it/s]
2it [00:01,  1.41it/s]
3it [00:02,  1.44it/s]
4it [00:02,  1.43it/s]
5it [00:03,  1.44it/s]
6it [00:04,  1.44it/s]
7it [00:04,  1.44it/s]
8it [00:05,  1.45it/s]
9it [00:06,  1.45it/s]
10it [00:06,  1.45it/s]
11it [00:07,  1.44it/s]
12it [00:08,  1.44it/s]
13it [00:09,  1.44it/s]
14it [00:09,  1.44it/s]
15it [00:10,  1.44it/s]
16it [00:11,  1.44it/s]
17it [00:11,  1.44it/s]
18it [00:12,  1.44it/s]
19it [00:13,  1.44it/s]
20it [00:13,  1.44it/s]
21it [00:14,  1.44it/s]
22it [00:15,  1.44it/s]
23it [00:15,  1.44it/s]
24it [00:16,  1.44it/s]
25it [00:17,  1.43it/s]
26it [00:18,  1.43it/s]
27it [00:18,  1.42it/s]
28it [00:19,  1.43it/s]
29it [00:20,  1.43it/s]
30it [00:20,  1.43it/s]
31it [00:21,  1.43it/s]
32it [00:22,  1.43it/s]
33it [00:22,  1.43it/s]
34it [00:23,  1.43it/s]
35it [00:24,  1.42it/s]
36it [00:25,  1.42it/s]
37it [00:25,  1.42it/s]
38it [00:26,  1.42it/s]
39it [00:27,  1.42it/s]
40it [00:27,  1.42it/s]
41it [00:28,  1.42it/s]
42it 

...

Checkpoint Model Saved!

Epoch: 03
Train Total Loss: 0.090 | Train F1 Score: 0.975
Valid Total Loss: 0.066 | Valid F1 Score: 0.984
--------------------



0it [00:00, ?it/s]
1it [00:00,  1.34it/s]
2it [00:01,  1.37it/s]
3it [00:02,  1.38it/s]
4it [00:02,  1.39it/s]
5it [00:03,  1.39it/s]
6it [00:04,  1.39it/s]
7it [00:05,  1.40it/s]
8it [00:05,  1.40it/s]
9it [00:06,  1.40it/s]
10it [00:07,  1.40it/s]
11it [00:07,  1.39it/s]
12it [00:08,  1.39it/s]
13it [00:09,  1.39it/s]
14it [00:10,  1.39it/s]
15it [00:10,  1.39it/s]
16it [00:11,  1.39it/s]
17it [00:12,  1.39it/s]
18it [00:12,  1.38it/s]
19it [00:13,  1.39it/s]
20it [00:14,  1.38it/s]
21it [00:15,  1.38it/s]
22it [00:15,  1.38it/s]
23it [00:16,  1.39it/s]
24it [00:17,  1.38it/s]
25it [00:18,  1.38it/s]
26it [00:18,  1.38it/s]
27it [00:19,  1.38it/s]
28it [00:20,  1.38it/s]
29it [00:20,  1.38it/s]
30it [00:21,  1.37it/s]
31it [00:22,  1.37it/s]
32it [00:23,  1.38it/s]
33it [00:23,  1.38it/s]
34it [00:24,  1.38it/s]
35it [00:25,  1.38it/s]
36it [00:26,  1.37it/s]
37it [00:26,  1.37it/s]
38it [00:27,  1.37it/s]
39it [00:28,  1.37it/s]
40it [00:28,  1.38it/s]
41it [00:29,  1.38it/s]
42it 

Checkpoint Model Saved!

Epoch: 04
Train Total Loss: 0.061 | Train F1 Score: 0.984
Valid Total Loss: 0.083 | Valid F1 Score: 0.982
--------------------



0it [00:00, ?it/s]
1it [00:00,  1.48it/s]
2it [00:01,  1.42it/s]
3it [00:02,  1.39it/s]
4it [00:02,  1.40it/s]
5it [00:03,  1.39it/s]
6it [00:04,  1.39it/s]
7it [00:05,  1.39it/s]
8it [00:05,  1.39it/s]
9it [00:06,  1.39it/s]
10it [00:07,  1.39it/s]
11it [00:07,  1.39it/s]
12it [00:08,  1.39it/s]
13it [00:09,  1.39it/s]
14it [00:10,  1.39it/s]
15it [00:10,  1.39it/s]
16it [00:11,  1.39it/s]
17it [00:12,  1.38it/s]
18it [00:12,  1.39it/s]
19it [00:13,  1.39it/s]
20it [00:14,  1.39it/s]
21it [00:15,  1.39it/s]
22it [00:15,  1.39it/s]
23it [00:16,  1.39it/s]
24it [00:17,  1.38it/s]
25it [00:17,  1.38it/s]
26it [00:18,  1.38it/s]
27it [00:19,  1.38it/s]
28it [00:20,  1.38it/s]
29it [00:20,  1.37it/s]
30it [00:21,  1.38it/s]
31it [00:22,  1.38it/s]
32it [00:23,  1.38it/s]
33it [00:23,  1.38it/s]
34it [00:24,  1.38it/s]
35it [00:25,  1.38it/s]
36it [00:25,  1.38it/s]
37it [00:26,  1.38it/s]
38it [00:27,  1.38it/s]
39it [00:28,  1.38it/s]
40it [00:28,  1.38it/s]
41it [00:29,  1.38it/s]
42it 

Checkpoint Model Saved!

Epoch: 05
Train Total Loss: 0.047 | Train F1 Score: 0.988
Valid Total Loss: 0.076 | Valid F1 Score: 0.986
--------------------


In [29]:
print(classification_report(best_target, best_pred))

              precision    recall  f1-score   support

           0       0.98      0.99      0.99       247
           1       0.98      0.97      0.98       180
           2       0.99      0.99      0.99        73

    accuracy                           0.98       500
   macro avg       0.98      0.98      0.98       500
weighted avg       0.98      0.98      0.98       500



In [30]:
from sklearn.metrics import f1_score
f1_score(best_target, best_pred, average='micro')

0.984

In [31]:
from google.colab import drive

In [32]:
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [33]:
from glob import glob

In [34]:
for filepath in glob("*.pt"):
    !cp -r $filepath /content/gdrive/My\ Drive/Colab\ Notebooks/NLP\ Final\ Project/
    time.sleep(10)

In [ ]:
# !cp -r model_least_loss_rob.pt /content/gdrive/My\ Drive/Colab\ Notebooks/NLP\ Final\ Project/

In [ ]:
# with open('/content/gdrive/My Drive/', 'w') as handle:
#     handle.write()